In [116]:
import math
import numpy as np

def get_plane(A, B, C):
    n = np.cross(B - A , C - A)
    d = -(n[0] * A[0] + n[1] * A[1] + n[2] * A[2])
    return (n, d)

def mirror_point(a, b, c, d, point):
    x1, y1, z1 = point
    k =(-a * x1-b * y1-c * z1-d)/float((a * a + b * b + c * c))
    x2 = a * k + x1
    y2 = b * k + y1
    z2 = c * k + z1
    x3 = 2 * x2-x1
    y3 = 2 * y2-y1
    z3 = 2 * z2-z1
    return x3, y3, z3

In [117]:
class Tetra:
    
    def __init__(self, sA, sB, sC, sD):
        self.A = sA
        self.B = sB
        self.C = sC
        self.D = sD
        
    def center(self):
        return ((self.A[0] + self.B[0] + self.C[0] + self.D[0])/4, 
        (self.A[1] + self.B[1] + self.C[1] + self.D[1])/4, 
        (self.A[2] + self.B[2] + self.C[2] + self.D[2])/4)
    
    def reflect(self, Vertex):
        
        if Vertex == "A":
            plane = get_plane(self.B, self.C, self.D)
            new_vertex = mirror_point(*plane[0], plane[1], self.A)
            vertices = (new_vertex, self.B, self.C, self.D)
        elif Vertex == "B":
            plane = get_plane(self.A, self.C, self.D)
            new_vertex = mirror_point(*plane[0], plane[1], self.B)
            vertices = (self.A, new_vertex, self.C, self.D)
        elif Vertex == "C":
            plane = get_plane(self.A, self.B, self.D)
            new_vertex = mirror_point(*plane[0], plane[1], self.C)
            vertices = (self.A, self.B, new_vertex, self.D)
        elif Vertex == "D":
            plane = get_plane(self.A, self.B, self.C)
            new_vertex = mirror_point(*plane[0], plane[1], self.D)
            vertices = (self.A, self.B, self.C, new_vertex)
            
        return Tetra(*vertices)
            
    def reflect_seq(self, seq):
        if len(seq) == 1:
            return self.reflect(seq)
        else:
            return self.reflect(seq[0]).reflect_seq(seq[1:])
            
       

In [118]:
# Define first tetrahedron with edges
t1 = Tetra(np.array((0,0,0)), np.array((math.sqrt(2), 0, -1)), np.array((0, math.sqrt(2), -1)), np.array((0,0,-1)))

# get the center of mass of the tetrahedron
print(t1.center())

# get a new tetrahedron by reflection the first one through edge C
t2 = t1.reflect("C")

# get the center of the tetrahedron which is created by sequentially reflecting tetrahedron 1 by the Sequence AA
# (first reflect t1 by A, and then reflect the result by A, and get the center of the final result)
print(t1.reflect_seq("AA").center())

(0.3535533905932738, 0.3535533905932738, -0.75)
(0.3535533905932738, 0.3535533905932738, -0.75)
